In [ ]:
select * from company_revenue

In [ ]:
with cte as
(
SELECT
*,
lead(revenue) OVER(PARTITION BY YEAR ORDER BY revenue) as inc,
lag(revenue) OVER(PARTITION BY YEAR ORDER BY revenue) as dec
from company_revenue
)
select company from cte
where inc > dec
GROUP BY company


In [ ]:
--first solution
with cte as
(
SELECT
*,
lag(revenue,1,0) over(PARTITION BY company ORDER BY year) as prev_rev,
revenue - lag(revenue,1,0) over(PARTITION BY company ORDER BY year) as revenue_diff,
count(1) over(PARTITION BY company) as cnt
from company_revenue
)
select company, cnt, count(1) as sales_inc
from cte
where revenue_diff > 0
group by company, cnt
having cnt = count(1)

In [ ]:
--second solution
with cte as
(
SELECT
*,
--lag(revenue,1,0) over(PARTITION BY company ORDER BY year) as prev_rev,
revenue - lag(revenue,1,0) over(PARTITION BY company ORDER BY year) as revenue_diff
--count(1) over(PARTITION BY company) as cnt
from company_revenue
)
select company --cnt, count(1) as sales_inc
from cte
where company not in (select company from cte where revenue_diff<0)
group by company
